In [1]:
import torch
import torch.nn.functional as F

In [2]:
test = torch.tensor([[[[ 9.4619e-01,  1.6403e+00, -1.9792e-01,  6.3207e-01,  2.0947e-01],
          [-1.6187e-01, -4.1497e-01, -9.9845e-01,  1.4859e+00, -1.6829e+00],
          [-1.3136e-02, -9.0405e-01,  1.1327e+00,  3.4932e-01,  3.2573e+00],
          [-2.3403e+00,  1.4493e+00,  1.3915e-01,  3.3621e-01,  8.2049e-01],
          [-6.3989e-01,  1.6800e+00,  2.1310e-01, -1.1094e+00,  1.0873e+00]],

         [[ 4.9299e-01, -1.3245e+00, -1.5166e-01, -8.6045e-01, -8.5921e-01],
          [-1.2423e+00, -1.5187e-01,  7.5842e-01,  7.4492e-01, -3.0696e-02],
          [-1.3522e+00, -4.2280e-01,  1.9291e-01, -4.3411e-01, -7.4983e-01],
          [ 3.2116e-01, -3.2613e-01,  2.1321e-01,  6.7839e-02, -3.2492e-01],
          [-8.1486e-04,  2.3210e-03,  7.4757e-01, -4.9818e-01, -1.3682e-01]]]])

In [3]:
test

tensor([[[[ 9.4619e-01,  1.6403e+00, -1.9792e-01,  6.3207e-01,  2.0947e-01],
          [-1.6187e-01, -4.1497e-01, -9.9845e-01,  1.4859e+00, -1.6829e+00],
          [-1.3136e-02, -9.0405e-01,  1.1327e+00,  3.4932e-01,  3.2573e+00],
          [-2.3403e+00,  1.4493e+00,  1.3915e-01,  3.3621e-01,  8.2049e-01],
          [-6.3989e-01,  1.6800e+00,  2.1310e-01, -1.1094e+00,  1.0873e+00]],

         [[ 4.9299e-01, -1.3245e+00, -1.5166e-01, -8.6045e-01, -8.5921e-01],
          [-1.2423e+00, -1.5187e-01,  7.5842e-01,  7.4492e-01, -3.0696e-02],
          [-1.3522e+00, -4.2280e-01,  1.9291e-01, -4.3411e-01, -7.4983e-01],
          [ 3.2116e-01, -3.2613e-01,  2.1321e-01,  6.7839e-02, -3.2492e-01],
          [-8.1486e-04,  2.3210e-03,  7.4757e-01, -4.9818e-01, -1.3682e-01]]]])

In [4]:
F.sigmoid(test)

tensor([[[[0.7203, 0.8376, 0.4507, 0.6530, 0.5522],
          [0.4596, 0.3977, 0.2692, 0.8155, 0.1567],
          [0.4967, 0.2882, 0.7563, 0.5865, 0.9629],
          [0.0878, 0.8099, 0.5347, 0.5833, 0.6943],
          [0.3453, 0.8429, 0.5531, 0.2480, 0.7479]],

         [[0.6208, 0.2101, 0.4622, 0.2972, 0.2975],
          [0.2240, 0.4621, 0.6810, 0.6781, 0.4923],
          [0.2055, 0.3958, 0.5481, 0.3931, 0.3209],
          [0.5796, 0.4192, 0.5531, 0.5170, 0.4195],
          [0.4998, 0.5006, 0.6786, 0.3780, 0.4658]]]])